In [ ]:
def fetchOHLC(ticker,interval,range_from, range_to):
    data = {
        "symbol":ticker,
        "resolution":interval,
        "date_format":"1",
        "range_from":range_from,
        "range_to":range_to,
        "cont_flag":"1"
    }

    '''
    1 minute : “1”
    2 minute : “2"
    3 minute : "3"
    5 minute : "5"
    10 minute : "10"
    15 minute : "15"
    20 minute : "20"
    30 minute : "30"
    60 minute : "60"
    120 minute : "120"
    240 minute : "240"
    Daily : "D"
    range format: yyyy-mm-dd
    '''
    response = fyers.history(data=data)['candles']

    # Create a DataFrame
    columns = ['Timestamp','Open','High','Low','Close','Volume']
    df = pd.DataFrame(response, columns=columns)

    # Convert Timestamp to datetime in UTC
    df['Timestamp2'] = pd.to_datetime(df['Timestamp'],unit='s').dt.tz_localize(pytz.utc)

    # Convert Timestamp to IST
    ist = pytz.timezone('Asia/Kolkata')
    df['Timestamp2'] = df['Timestamp2'].dt.tz_convert(ist)

    return (df)

# Fetch OHLC data using the function
response_df = fetchOHLC("NSE:SBIN-EQ","15","2023-08-18", "2023-08-29")

# Print the DataFrame
print(response_df)

# Save data to a CSV file
response_df.to_csv('output.csv', index=False)

In [ ]:
import pandas as pd
import requests
import time
from datetime import datetime, timedelta, date
from fyers_apiv3 import fyersModel


# 📥 Load your uploaded Nifty 500 list
def load_nifty500_symbols():
    df = pd.read_csv("ind_nifty500list.csv")  # Must be in same folder
    df['fyers_symbol'] = "NSE:" + df['Symbol'] + "-" + df['Series']
    return df['fyers_symbol'].tolist()


# 📊 Camarilla calculation
def calculate_camarilla_levels(high, low, close):
    range_ = high - low
    return {
        "L1": close - (range_ * 1.1 / 12),
        "H1": close + (range_ * 1.1 / 12),
        "L4": close - (range_ * 1.1 / 2),
        "H4": close + (range_ * 1.1 / 2),
        "L5": close - ((high / low) * close - close),
        "H5": (high / low) * close,
    }


# 📈 Fetch historical data from Fyers
def get_history(symbol, from_date, to_date, resolution="D"):
    payload = {
        "symbol": symbol,
        "resolution": resolution,
        "date_format": "1",
        "range_from": from_date,
        "range_to": to_date,
        "cont_flag": "1"
    }
    response = fyers.history(payload)
    if response.get("s") != "ok" or "candles" not in response:
        return None
    df = pd.DataFrame(response["candles"], columns=["timestamp", "open", "high", "low", "close", "volume"])
    # Convert Timestamp to datetime in UTC
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s").dt.tz_localize(pytz.utc)

     # Convert Timestamp to IST
    ist = pytz.timezone('Asia/Kolkata')
    df['timestamp'] = df['timestamp'].dt.tz_convert(ist)
    
    df["date"] = df["timestamp"].dt.date
    df["time"] = df["timestamp"].dt.time
    return df


# 🔍 Analyze one symbol
def analyze_symbol(symbol, today):
    try:
        # Step 1: Daily data
        from_date = (today - timedelta(days=7)).strftime("%Y-%m-%d")
        to_date = today.strftime("%Y-%m-%d")
        daily_df = get_history(symbol, from_date, to_date, resolution="D")
        
        if daily_df is None or len(daily_df) < 2:
            return None

        prev_day = daily_df.iloc[-2]
        camarilla = calculate_camarilla_levels(prev_day["high"], prev_day["low"], prev_day["close"])
    
        # Step 2: Intraday 15-min data for today
        intraday_df = get_history(symbol, today.strftime("%Y-%m-%d"), today.strftime("%Y-%m-%d"), resolution="15")
        #intraday_df = get_history(symbol, "2025-10-03", "2025-10-03", resolution="15")
        
        if intraday_df is None or intraday_df.empty:
            return None

        # Get 09:15 candle
        first_candle = intraday_df[intraday_df["time"] == datetime.strptime("09:15", "%H:%M").time()]
        if first_candle.empty:
            return None

        row = first_candle.iloc[0]
        support_broken = row["high"] >= camarilla["H1"] and row["close"] < camarilla["L1"]
        return {
            "symbol": symbol,
            "prev_high": prev_day["high"],
            "prev_low": prev_day["low"],
            "prev_close": prev_day["close"],
            "first_high": row["high"],
            "first_close": row["close"],
            **camarilla,
            "support_broken": support_broken
        }
    except Exception as e:
        print(f"Error for {symbol}: {e}")
        return None


# 🧠 Main loop
def run_camarilla_scan():
    today = date.today()

    symbols = load_nifty500_symbols()
    #print(f"🔎 Loaded {len(symbols)} Nifty 500 symbols.")

    results = []
    for i, symbol in enumerate(symbols):
        #print(f"{i + 1}/{len(symbols)}: {symbol}")
        result = analyze_symbol(symbol, today)
        if result:
            results.append(result)
        time.sleep(0.02)  # Sleep to avoid rate limiting

    df = pd.DataFrame(results)
    # Filter for stocks where support is broken
  
    support_broken_df = df[df["support_broken"] == True]

    # Save both full and filtered results
    df.to_csv("nifty500_camarilla_results.csv", index=False)
    support_broken_df.to_csv("support_broken_stocks.csv", index=False)
    
    #Print support-broken stock symbols
    if not support_broken_df.empty:
        print("Stocks where support is broken in first 15-min candle:")
        for sym in support_broken_df["symbol"]:
            print(f"{sym}")
    else:
        print("No stocks triggered support-broken condition today.")

# ▶️ Run
if __name__ == "__main__":
    run_camarilla_scan()